In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


import os
import sys
from torchsummary import summary


# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/

#device config
torch.cuda.set_device(1)#Select GPU device 1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Get Packages
sys.path.append('./Tools')

#Network architectures
from ResNet18_CIFAR10 import ResNet18_CIFAR10_Model
from ResNet18_TINYIMAGENET import ResNet18_TINYIMAGENET_Model
from VGG16_CIFAR10 import VGG16_CIFAR10_Model
from VGG16_TINYIMAGENET import VGG16_TINYIMAGENET_Model

#Non Corrupted DataLoaders
from CIFAR10_LOADER import CIFAR10DataLoader
from TINYIMAGENET_LOADER import TINYIMAGENETDataLoader

#Error Metrics
from ErrorMetrics import Evaluate_Model_TOP1


# Unpruned Models

In [2]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0.pt")
ResNet18_TINYIMAGENET_Weights=torch.load("./Networks/ResNet18_TINY_IMAGENET_V0.pt")
VGG16_CIFAR10_Weights=torch.load("./Networks/VGG16_CIFAR10_V0.pt")
VGG16_TINYIMAGENET_Weights=torch.load("./Networks/VGG16_TINY_IMAGENET_V0.pt")

#Unpruned Models
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

VGG16_CIFAR10_0P=VGG16_CIFAR10_Model()
VGG16_CIFAR10_0P.load_state_dict(VGG16_CIFAR10_Weights)

VGG16_TINYIMAGENET_0P=VGG16_TINYIMAGENET_Model()
VGG16_TINYIMAGENET_0P.load_state_dict(VGG16_TINYIMAGENET_Weights)


<All keys matched successfully>

# Load Datasets

In [3]:
trainloader_CIFAR10, testloader_CIFAR10=CIFAR10DataLoader() #Load Data for CIFAR10
trainloader_TINYIMAGENET, testloader_TINYIMAGENET=TINYIMAGENETDataLoader() #Load Data for TINYIMAGENET

Files already downloaded and verified
Files already downloaded and verified


# Evaluate Non Corrupted Data Models

In [5]:

#Test Accuracy
loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_CIFAR10_0P, device, testloader_CIFAR10) #ResNet18 on CIFAR10
print(f"ResNet18_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_TINYIMAGENET_0P, device, testloader_TINYIMAGENET) #ResNet18 on TINYIMAGENET
print(f"ResNet18_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_CIFAR10_0P, device, testloader_CIFAR10) #VGG16 on CIFAR10
print(f"VGG16_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_TINYIMAGENET_0P, device, testloader_TINYIMAGENET) #VGG16 on TINYIMAGENET
print(f"VGG16_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")


100%|██████████| 20/20 [00:01<00:00, 10.74it/s]


ResNet18_CIFAR10_0P: Loss: 0.3436, Accuracy: 89.04%




100%|██████████| 20/20 [00:02<00:00,  9.18it/s]


ResNet18_TINYIMAGENET_0P: Loss: 1.7581, Accuracy: 60.36%




100%|██████████| 20/20 [00:01<00:00, 10.43it/s]


VGG16_CIFAR10_0P: Loss: 0.3662, Accuracy: 89.09%




100%|██████████| 20/20 [00:01<00:00, 10.18it/s]

VGG16_TINYIMAGENET_0P: Loss: 2.7202, Accuracy: 54.76%




In [7]:
#Train Accuracy
loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_CIFAR10_0P, device, trainloader_CIFAR10) #ResNet18 on CIFAR10
print(f"ResNet18_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_TINYIMAGENET_0P, device, trainloader_TINYIMAGENET) #ResNet18 on TINYIMAGENET
print(f"ResNet18_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_CIFAR10_0P, device, trainloader_CIFAR10) #VGG16 on CIFAR10
print(f"VGG16_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_TINYIMAGENET_0P, device, trainloader_TINYIMAGENET) #VGG16 on TINYIMAGENET
print(f"VGG16_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

100%|██████████| 98/98 [00:19<00:00,  4.91it/s]


ResNet18_CIFAR10_0P: Loss: 0.1349, Accuracy: 95.64%




100%|██████████| 391/391 [01:00<00:00,  6.43it/s]


ResNet18_TINYIMAGENET_0P: Loss: 0.5043, Accuracy: 87.22%




100%|██████████| 98/98 [00:19<00:00,  4.93it/s]


VGG16_CIFAR10_0P: Loss: 0.1292, Accuracy: 95.64%




100%|██████████| 391/391 [01:01<00:00,  6.38it/s]

VGG16_TINYIMAGENET_0P: Loss: 0.0141, Accuracy: 99.78%


